In [ ]:
import os, sys
from datetime import datetime, timedelta
import pandas as pd
import ipywidgets as widgets
import plotly.graph_objs as go
import yfinance as yf
import pandas as pd
from IPython.display import display

In [ ]:
full_path = os.path.abspath(os.path.join(os.getcwd(),"..", "..", "custom_imports", "dashboards", "futures.csv"))
df = pd.read_csv(full_path)
months = ["F", "G", "H", "J", "K", "M", "N", "Q", "U", "V", "X", "Z"]

In [ ]:
import os, sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def format_plotly(fig, data, chart, calc=None):
    fig.update_yaxes(title=None)
    fig.update_xaxes(title=None)
    if calc:
        if len(calc) == 1:
            fig_title = f"{calc[0]} of {data}"
        else:
            fig_title = f"{', '.join(calc)} of {data}"
    else:
        fig_title = "Volume"
    height = 500 if chart == "main" else 300
    fig.update_layout(
        margin=dict(l=0, r=10, t=10, b=10),
        autosize=False,
        width=900,
        height=height,
        legend=dict(orientation="h"),
        title={
            "text": fig_title,
            "y": 0.95,
            "x": 0.5,
            "xanchor": "center",
            "yanchor": "top",
        },
    )


def create_line(visual, x, y, name, data, fig):
    if visual == "line":
        plot = go.Scatter(x=x, y=y[data], mode="lines", name=name, connectgaps=True)
    if visual == "scatter":
        plot = go.Scatter(x=x, y=y[data], mode="markers", name=name)
    if visual == "candle":
        plot = go.Candlestick(
            x=x,
            open=y["Open"],
            close=y["Close"],
            high=y["High"],
            low=y["Low"],
            name=name,
        )
    fig.add_trace(plot)


def show_fig(fig):
    config = {"showTips": False, "scrollZoom": True}
    if os.environ.get("SERVER_SOFTWARE", "jupyter").startswith("voila"):
        fig.show(config=config, renderer="notebook")
    else:
        fig.show(config=config)

def build_ticker(ticker: str, month: int, year: int) -> str:
    if ticker:
        row = df[df["Ticker"] == f"{ticker.split(':')[0]}=F"].iloc[0]
        row = row.to_dict()
        the_tick = row['Ticker'].replace('=F','')
        return f"{the_tick}{months[month-1]}{str(year)[-2:]}.{row['Exchange']}"
    return ""

def next_ticker(ticker: str) -> str:
    print(ticker)
    symbol, exchange = ticker.split(".")
    month = symbol[-3]
    if month == "Z":
        new_month = "F"
        new_year = int(symbol[-2:]) + 1
    else:
        index = months.index(month)
        new_month = months[index + 1]
        new_year = int(symbol[-2:])
    return f"{symbol[:-3]}{new_month}{new_year}.{exchange}"

def get_column(df: pd.DataFrame, column: str):
    sub_df = df.xs(column, level=1, axis=1, drop_level=False)
    sub_df.columns = sub_df.columns.to_flat_index().map(lambda x: x[0])
    return sub_df

In [ ]:
class Chart:
    def __init__(self):
        self.tickers = {}

    def create_stock(self, chart_type, contracts, ticker):
        if not ticker:
            return

        now = datetime.now()
        clean_ticker = build_ticker(ticker, now.month, now.year)
        if clean_ticker:
            raw_tickers = [clean_ticker]
            for _ in range(12):
                new_ticker = next_ticker(raw_tickers[-1])
                raw_tickers.append(new_ticker)
                raw_ticker = ",".join(raw_tickers)
            with HiddenPrints():
                dfs = yf.download(raw_ticker, progress=False) # ,period="max"
            self.tickers = {x: get_column(dfs, x) for x in raw_tickers}
            print(self.tickers)

        fig = go.Figure()
        if result.empty:
            return

        if len(result.columns) == 6:
            name = clean_ticker
            create_line("line", result.index, result, name, "Adj Close", fig)

        else:
            for val in result.columns.levels[1]:
                vals = result.xs(val, axis=1, level=1, drop_level=True)
                name = clean_ticker
                create_line("line", result.index, vals, name, "Adj Close", fig)

        format_plotly(fig, "Close", "main", "raw")
        show_fig(fig)

In [ ]:
category = df["Category"].unique().tolist()
exchange = df[df["Category"].isin([category[0]])]["Exchange"].unique().tolist()
filt_df = df[df["Category"].isin([category[0]]) & df["Exchange"].isin([exchange[0]])]
tickers_raw = filt_df[["Ticker", "Description"]].values.tolist()
tickers = [f"{x.replace('=F', '')}: {y}" for x, y in tickers_raw]

chart_type = ["Historical Time Series", "Future Curve"]
cat_widget = widgets.SelectMultiple(options=category, value=[category[0]])
exch_widget = widgets.SelectMultiple(options=exchange, value=[exchange[0]])
tickers_widget = widgets.Select(options=tickers, value=tickers[0])
chart_widget = widgets.SelectMultiple(options=chart_type, value=[chart_type[0]])

def on_change(change):
    cat_value = cat_widget.value
    exch_value = exch_widget.value
    
    #Filter exchange widget
    exch_filtered = df[df["Category"].isin(cat_value)]["Exchange"].unique().tolist()
    exch_widget.options = exch_filtered
    
    #Filter ticker widget
    filtered = df[df["Category"].isin(cat_value) & df["Exchange"].isin(exch_value)]
    tick_raw = filtered[["Ticker", "Description"]].values.tolist()
    tick = [f"{x.replace('=F', '')}: {y}" for x, y in tick_raw]
    tickers_widget.options = tick
    
cat_widget.observe(on_change)
exch_widget.observe(on_change)

contracts_widget = widgets.Dropdown(options=list(range(1, 13)), value=6, description="Contracts")

controls = widgets.HBox(
    [cat_widget, exch_widget, tickers_widget, chart_widget, contracts_widget],
    layout=widgets.Layout(width="90%"),
)
chart = Chart()
stocks_view = widgets.interactive_output(
    chart.create_stock,
    {
        "chart_type": chart_widget,
        "contracts": contracts_widget,
        "ticker": tickers_widget,
    },
)


title_html = "<h1>Stock Analysis Dashboard</h1>"

app_contents = [widgets.HTML(title_html), controls, stocks_view]
app = widgets.VBox(app_contents)
display(app)

In [ ]:
pd.set_option('display.max_rows', None)
chart.tickers['YIN23.NYB'].sort_values(by=['Date']).to_csv("YIN23_NYB.csv")